# I14Y Concepts to LINDAS Defined Term Sets

In [2]:
from bequtils.nb.rdf import *

set_display_options()

✅ Pandas display options set for full visibility.


## Show All Defined Term Sets

Not the single versions but only the identities.

In [9]:
query = """

PREFIX schema: <http://schema.org/>
PREFIX vl: <https://version.link/>

SELECT ?concept FROM <https://lindas.admin.ch/fso/i14y> WHERE {
  
  ?concept a schema:DefinedTermSet ;
    a vl:Identity .
}
"""

data = query_endpoint("https://ld.admin.ch/query", query)

display_result(data)

,concept
0,https://register.ld.admin.ch/i14y/concept/BEWOHNERTYP
1,https://register.ld.admin.ch/i14y/concept/DV_AUSLAENDERKATEGORIEN
2,https://register.ld.admin.ch/i14y/concept/DV_AUSWEISTYP
3,https://register.ld.admin.ch/i14y/concept/DV_SEXE
4,https://register.ld.admin.ch/i14y/concept/ETAT_CIVIL
5,https://register.ld.admin.ch/i14y/concept/UMZUG
6,https://register.ld.admin.ch/i14y/concept/VOCAB_I14Y_ATTRIBUTION_ROLE
7,https://register.ld.admin.ch/i14y/concept/legalForm
8,https://register.ld.admin.ch/i14y/concept/VOCAB_EU_DATA_THEME
9,https://register.ld.admin.ch/i14y/concept/VOCAB_EU_FREQUENCY


## Show Predicates and Objects

For a single Defined Term Set.

In [10]:
query = """
PREFIX schema: <http://schema.org/>
SELECT ?p ?o
WHERE {
  <https://register.ld.admin.ch/i14y/concept/legalForm> ?p ?o .
}
"""

data = query_endpoint("https://ld.admin.ch/query", query)

display_result(data)

,p,o
0,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://schema.org/DefinedTermSet
1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2004/02/skos/core#ConceptScheme
2,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,https://version.link/Identity
3,http://purl.org/dc/terms/identifier,legalForm
4,http://www.w3.org/2004/02/skos/core#prefLabel,Rechtsform eines Unternehmens (identity)
5,http://www.w3.org/2004/02/skos/core#prefLabel,Legal form of a company (identity)
6,http://www.w3.org/2004/02/skos/core#prefLabel,Forme juridique d'une entreprise (identity)
7,http://www.w3.org/2004/02/skos/core#prefLabel,Forma giuridica di una società (identity)
8,http://purl.org/dc/terms/title,Rechtsform eines Unternehmens (identity)
9,http://purl.org/dc/terms/title,Legal form of a company (identity)


## Start and Stop Dates of Versions

In [11]:
query = """

PREFIX schema: <http://schema.org/>
SELECT ?concept ?validFrom ?validThrough
WHERE {
  GRAPH <https://lindas.admin.ch/fso/i14y> {
    ?concept a schema:DefinedTermSet , <https://version.link/Version> .
    OPTIONAL {?concept schema:validFrom ?validFrom .}
    OPTIONAL {?concept schema:validThrough ?validThrough .}
  }
}

"""
data = query_endpoint("https://ld.admin.ch/query", query)
display_result(data)

,concept,validFrom,validThrough
0,https://register.ld.admin.ch/i14y/concept/BEWOHNERTYP/version/1.0.0,2020-09-13,None
1,https://register.ld.admin.ch/i14y/concept/DV_AUSLAENDERKATEGORIEN/version/1.0.0,2009-12-31,None
2,https://register.ld.admin.ch/i14y/concept/DV_AUSWEISTYP/version/1.0.0,2009-12-31,None
3,https://register.ld.admin.ch/i14y/concept/DV_SEXE/version/2.0.0,2022-03-30,None
4,https://register.ld.admin.ch/i14y/concept/ETAT_CIVIL/version/1.0.0,2021-07-06,None
5,https://register.ld.admin.ch/i14y/concept/UMZUG/version/1.0.0,2011-12-31,None
6,https://register.ld.admin.ch/i14y/concept/VOCAB_I14Y_ATTRIBUTION_ROLE/version/1.0.0,2023-06-14,None
7,https://register.ld.admin.ch/i14y/concept/legalForm/version/1.1.0,2010-12-31,None
8,https://register.ld.admin.ch/i14y/concept/VOCAB_EU_DATA_THEME/version/20220715.0.0,2022-07-15,None
9,https://register.ld.admin.ch/i14y/concept/VOCAB_EU_FREQUENCY/version/20230614.0.0,2023-06-14,None


--> They all have a schema:validFrom but no schema:validThrough (even if they have a successor)

## Datacite Vocabulary

In [12]:
query = """
SELECT DISTINCT ?p WHERE {
    GRAPH <https://lindas.admin.ch/fso/i14y> {
        ?s ?p ?o .
    }
    FILTER(CONTAINS(STR(?p), "datacite"))
}
"""

data = query_endpoint("https://ld.admin.ch/query", query)
display_result(data)

,p
0,https://datacite-metadata-schema.readthedocs.io/en/4.6/appendices/appendix-1/relationType/#haspart


--> Seems not to be the correct uri for datacite

## Version Link Vocabulary

In [13]:
query = """
SELECT DISTINCT ?p WHERE {
    GRAPH <https://lindas.admin.ch/fso/i14y> {
        ?s ?p ?o .
    }
    FILTER(CONTAINS(STR(?p), "version.link"))
}
"""
data = query_endpoint("https://ld.admin.ch/query", query)
display_result(data)

,p
0,https://version.link/identity
1,https://version.link/version
2,https://version.link/successor
3,https://version.link/predecessor


In [14]:
query = """
SELECT DISTINCT ?o WHERE {
    GRAPH <https://lindas.admin.ch/fso/i14y> {
        ?s ?p ?o .
    }
    FILTER(CONTAINS(STR(?o), "version.link"))
}
"""
data = query_endpoint("https://ld.admin.ch/query", query)
display_result(data)

,o
0,https://version.link/Version
1,https://version.link/Identity
2,https://version.link/Deprecated


## How Many Identities and Versions Are There?

In [15]:
query = """
SELECT (COUNT(?s) AS ?no) WHERE {
    GRAPH <https://lindas.admin.ch/fso/i14y> {
        ?s a <https://version.link/Identity> .
    }
}
"""
data = query_endpoint("https://ld.admin.ch/query", query)
display_result(data)

,no
0,2925


In [16]:
query = """
SELECT (COUNT(?s) AS ?no) WHERE {
    GRAPH <https://lindas.admin.ch/fso/i14y> {
        ?s a <https://version.link/Version> .
    }
}
"""
data = query_endpoint("https://ld.admin.ch/query", query)
display_result(data)

,no
0,4894


## Additional Data on Terms (Annotations)

In [17]:
query = """
PREFIX schema: <http://schema.org/>
SELECT * WHERE {
  <https://register.ld.admin.ch/i14y/concept/DV_KT_BEZ_GDE_SNAP/1> schema:hasPart ?part .
  ?part <https://www.w3.org/ns/oa#hasBody> ?body .
  ?body ?p ?o .
}
"""
data = query_endpoint("https://ld.admin.ch/query", query)
display_result(data)

,part,body,p,o


Questions:

- why so many duplications of information?
- where is the information for "HIER_LEVEL" (see: https://www.i14y.admin.ch/de/catalog/concepts/08dc23f0-d04d-2d2f-a9f5-9cea80695acf/content/8100/ZH/B_101/1)?

In [25]:
%%sparql df

SELECT * WHERE {
  <https://register.ld.admin.ch/i14y/concept/DV_KT_BEZ_GDE_SNAP/1> <http://www.w3.org/ns/shacl#property> ?shacl .
  ?shacl ?p ?o .
}

,shacl.type,shacl.value,p.type,p.value,o.type,o.value
0,bnode,anon-genid-37cbba4232c8401ab0faacd8da802561-15700,uri,http://www.w3.org/ns/shacl#path,uri,http://www.w3.org/2004/02/skos/core#notation
1,bnode,anon-genid-37cbba4232c8401ab0faacd8da802561-15700,uri,http://www.w3.org/ns/shacl#datatype,uri,http://www.w3.org/2001/XMLSchema#string
2,bnode,anon-genid-37cbba4232c8401ab0faacd8da802561-15701,uri,http://www.w3.org/ns/shacl#path,uri,http://www.w3.org/2004/02/skos/core#notation
3,bnode,anon-genid-37cbba4232c8401ab0faacd8da802561-15701,uri,http://www.w3.org/ns/shacl#datatype,uri,http://www.w3.org/2001/XMLSchema#string
4,bnode,anon-genid-b5f6f4f8800e435c9e169bc3b47dc9d5-15700,uri,http://www.w3.org/ns/shacl#path,uri,http://www.w3.org/2004/02/skos/core#notation
5,bnode,anon-genid-b5f6f4f8800e435c9e169bc3b47dc9d5-15700,uri,http://www.w3.org/ns/shacl#datatype,uri,http://www.w3.org/2001/XMLSchema#string
6,bnode,anon-genid-b5f6f4f8800e435c9e169bc3b47dc9d5-15701,uri,http://www.w3.org/ns/shacl#path,uri,http://www.w3.org/2004/02/skos/core#notation
7,bnode,anon-genid-b5f6f4f8800e435c9e169bc3b47dc9d5-15701,uri,http://www.w3.org/ns/shacl#datatype,uri,http://www.w3.org/2001/XMLSchema#string
8,bnode,anon-genid-b17e9d5b34c14ab7a787ee5542937c8b-15700,uri,http://www.w3.org/ns/shacl#path,uri,http://www.w3.org/2004/02/skos/core#notation
9,bnode,anon-genid-b17e9d5b34c14ab7a787ee5542937c8b-15700,uri,http://www.w3.org/ns/shacl#datatype,uri,http://www.w3.org/2001/XMLSchema#string


Questions:

- what are these information used for?

## Findings

### Blocking-Points

- missing /concept/ in URI --> https://register.ld.admin.ch/i14y/legalForm should be https://register.ld.admin.ch/i14y/concept/legalForm --> now corrected
- version.link URIs are wrong: e.g. https://version.link/#Identity should be https://version.link/Identity (all without #) --> now corrected

### Improvements

- a version that has a successor should have a stop date: e.g. https://register.ld.admin.ch/i14y/DV_ST/version/2023.1.0 should have schema:validThrough 2024-05-13 (because version 2024.1.0 has schema:validFrom 2024-05-14), it seems that at least for the DV_ST, there is also no "valid to" on I14Y (see https://www.i14y.admin.ch/de/catalog/concepts/08dbbb66-fb7b-1b35-ad15-80ff9c9bd177/description#section-versions)
- all versions should link via vl:identity to the corresponding identity (not only the newest version), e.g. https://register.ld.admin.ch/i14y/DV_ST/version/2023.1.0 should have vl:identity to <https://register.ld.admin.ch/i14y/DV_ST>, background is, that a version should never have a triple changed or removed (only added)
- I would propose that every concept from I14Y that has the class schema:DefinedTermSet should also have a <https://schema.ld.admin.ch/I14yConcept> class added to better separate it from all the other schema:DefinedTermSet on LINDAS (otherwise, the Query must always be narrowed down on the graph <https://lindas.admin.ch/fso/i14y>)
- usage of predicate https://datacite-metadata-schema.readthedocs.io/en/4.6/appendices/appendix-1/relationType/#haspart seems a little bit odd (it is the only predicate used that comes from this datacite vocabulary), I would propose to use skos:hasPart or xkos:hasPart as alternative
- concerning the /.well-known/void triples: Michael wrote an eMail on 13.06.2025 on this subject, I try to summarize: the single schema:DefinedTermSet do not need a triple schema:includedInDataCatalog <https://register.ld.admin.ch/i14y/.well-known/void> and <https://register.ld.admin.ch/i14y/.well-known/void> should not be used as subject, but the whole i14y dataset should be registered via schema:dataset to <https://register.ld.admin.ch/.well-known/dataset/i14y> in <https://register.ld.admin.ch/.well-known/void>, see the other datasets already present there --> so basically, the .well-known/void mechanism is not used to group all the single concepts into the "i14y dataset", but to register the "i14y dataset" as a whole on register.ld.admin.ch
- versions and identities should have a class vl:Version and vl:Identity respectively --> solved, maybe an error from my side that I did not check correctly

## Working with the Data

How to get a list with identifiers and labels for all languages?

In [33]:
query = """

PREFIX schema: <http://schema.org/>

SELECT ?term ?id ?isPartOf ?name_de ?name_fr ?name_it WHERE {
    BIND (<https://register.ld.admin.ch/i14y/concept/BEWOHNERTYP> AS ?concept)

    ?concept schema:hasDefinedTerm ?term .
    ?term schema:identifier ?id;
        schema:name ?name_de, ?name_fr, ?name_it .

    FILTER(lang(?name_de) = "de")
    FILTER(lang(?name_fr) = "fr")
    FILTER(lang(?name_it) = "it")

    OPTIONAL { ?term schema:isPartOf/schema:identifier ?isPartOf . }
}
"""
data = query_endpoint("https://ld.admin.ch/query", query)
display_result(data)

,term,id,isPartOf,name_de,name_fr,name_it
0,https://register.ld.admin.ch/i14y/concept/BEWOHNERTYP/30,30,None,Anderer Bewohnertyp,Autre statut d'occupation,Altro tipo di occupante
1,https://register.ld.admin.ch/i14y/concept/BEWOHNERTYP/10,10,None,Mieterin / Mieter oder Genossenschafterin / Genossenschafter,Locataire ou coopérateur/trice,Inquilino/a o socio/a di una cooperativa
2,https://register.ld.admin.ch/i14y/concept/BEWOHNERTYP/110,110,10,Mieterin / Mieter,Locataire,Inquilino/a
3,https://register.ld.admin.ch/i14y/concept/BEWOHNERTYP/120,120,10,Genossenschafterin / Genossenschafter,Coopérateur/trice,Socio/a di una cooperativa
4,https://register.ld.admin.ch/i14y/concept/BEWOHNERTYP/20,20,None,Eigentümerin / Eigentümer,Propriétaire,Proprietario/a
5,https://register.ld.admin.ch/i14y/concept/BEWOHNERTYP/210,210,20,Eigentümerin / Eigentümer des Hauses,Propriétaire de la maison,Proprietario/a della casa
6,https://register.ld.admin.ch/i14y/concept/BEWOHNERTYP/220,220,20,Stockwerk-/Wohnungseigentümerin / Stockwerk-/Wohnungseigentümer,Propriétaire du logement/propriétaire par étage,Proprietario/a dell'abitazione / proprietario/a per piani
